In [2]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from numba import njit

Standardize Data

In [31]:
df = pd.read_csv("Daily_201404180000_202503130000.csv",
                delim_whitespace=True,
                 names=['date', 'open', 'high', 'low', 'close', 'vol', 'del', 'spread'],
                 header=0)
df["date"] = pd.to_datetime(df["date"])
df.drop(columns="del", inplace=True)
df.set_index("date", inplace=True)
df.sort_index(inplace=True)
df.dropna(inplace=True)

column_to_convert = ['open', 'high', 'low', 'close', 'vol', 'spread']
df[column_to_convert] = df[column_to_convert].apply(pd.to_numeric, errors='coerce')
df.head()

C:\Users\najdu\AppData\Local\Temp\ipykernel_16336\4121226758.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("Daily_201404180000_202503130000.csv",


,open,high,low,close,vol,spread
date,,,,,,
2014-04-18,1294.39,1295.10,1294.30,1295.02,16,0
2014-04-21,1293.80,1301.28,1281.70,1289.41,197994,0
2014-04-22,1289.13,1292.80,1275.24,1283.41,139295,0
2014-04-23,1282.81,1288.99,1280.76,1283.36,120743,0
2014-04-24,1282.79,1298.58,1268.24,1292.80,221710,0
